In [131]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [132]:
import warnings
warnings.filterwarnings('ignore')

In [133]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [134]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [135]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [136]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaler.fit(df_market_data)
scaled_df = scaler.transform(df_market_data)

In [137]:
columns_list = df_market_data.columns.tolist()

In [138]:
# Create a DataFrame with the scaled data#
scaled_df = pd.DataFrame(scaled_df, columns = columns_list)
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
1,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
2,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
3,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
4,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


In [139]:
# Copy the crypto names from the original data
market_df = pd.DataFrame(index = df_market_data.index).reset_index()

# Set the coinid column as index
transformed_df = pd.concat([market_df, scaled_df], axis = 1)
transformed_df.set_index('coin_id', inplace = True)

# Display sample data
transformed_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [140]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [141]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state = 1, n_init = 10)
    k_model.fit(transformed_df)
    inertia.append(k_model.inertia_)

In [142]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {k[i]: inertia[i] for i in range(len(k))}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data.items(), columns = ['k','inertia'])

In [144]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x = 'k',
    y = 'inertia',
    title = 'Elbow Curve',
    label = 'Scaled'
)
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k = 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [14]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4, random_state = 1, n_init = 10)

In [15]:
# Fit the K-Means model using the scaled data
model.fit(transformed_df)

KMeans(n_clusters=4, n_init=10, random_state=1)

In [17]:
# Predict the clusters to group the cryptocurrencies using the scaled data
prediction = model.predict(transformed_df)

# Print the resulting array of cluster values.
prediction

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [18]:
# Create a copy of the DataFrame
transformed_prediction_df = transformed_df.copy()

In [19]:
# Add a new column to the DataFrame with the predicted clusters
transformed_prediction_df['k_Cluster'] = prediction

# Display sample data
transformed_prediction_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,k_Cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [89]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_plot=transformed_prediction_df.hvplot.scatter(
    x = "price_change_percentage_24h",
    y = "price_change_percentage_7d",
    by = "k_Cluster",
    hover_cols = "coin_id",
    title = "Cryptocurrency Clusters (k = 4)"
).opts(yformatter = "%.0f")
crypto_plot

:NdOverlay   [k_Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [24]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components = 3)

In [25]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca = pca_model.fit_transform(transformed_df)

# View the first five rows of the DataFrame. 
crypto_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [30]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
variance_ratio = pca_model.explained_variance_ratio_
print(variance_ratio)

[0.3719856  0.34700813 0.17603793]


In [31]:
# Add total explained variance three principal components
variance_ratio[0] + variance_ratio[1] + variance_ratio[2]

0.895031657030984

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three pricipal components is approximately 89.5%.

In [76]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    crypto_pca,
    columns = ["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
pca_df = pd.concat([market_df, pca_df], axis = 1)

# Set the coinid column as index
pca_df.set_index('coin_id', inplace = True)

# Display sample data
pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [145]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [146]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters = i, random_state = 1, n_init = 10)
    k_model.fit(pca_df)
    inertia.append(k_model.inertia_)

In [147]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {k[i]: inertia[i] for i in range(len(k))}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data.items(), columns = ['k','inertia'])

In [148]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow.hvplot.line(
    x = 'k',
    y = 'inertia',
    title = 'Elbow Curve (PCA)',
    label = 'Scaled + PCA'
)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k=4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** This is the same best k-value found when using the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [81]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4, random_state = 1, n_init = 10)

In [82]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

KMeans(n_clusters=4, n_init=10, random_state=1)

In [83]:
# Predict the clusters to group the cryptocurrencies using the PCA data
prediction_PCA = model.predict(pca_df)

# Print the resulting array of cluster values.
prediction_PCA

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0])

In [84]:
# Create a copy of the DataFrame with the PCA data
pca_prediction_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_prediction_df['k_Cluster_PCA'] = prediction_PCA

# Display sample data
pca_prediction_df.head()

,PC1,PC2,PC3,k_Cluster_PCA
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [85]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
crypto_plot_pca = pca_prediction_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = "k_Cluster_PCA",
    hover_cols = "coin_id",
    title = "Cryptocurrency Clusters (PCA = 3, k = 4)"
).opts(yformatter = "%.0f")
crypto_plot_pca

:NdOverlay   [k_Cluster_PCA]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [149]:
# Composite plot to contrast the Elbow curves
concat_plots = elbow_plot + elbow_plot_pca
concat_plots

:Layout
   .Curve.Scaled          :Curve   [k]   (inertia)
   .Curve.Scaled_plus_PCA :Curve   [k]   (inertia)

In [152]:
# Composite plot to contrast the clusters
# Added labels in original elbow line graphs so the legend would show up here

elbow_plot_pca.opts(line_dash = 'dotted')
plot_elbow_comp = elbow_plot * elbow_plot_pca
plot_elbow_comp = plot_elbow_comp.opts(title = "Elbow Curves Comparison")

# Display the plot
plot_elbow_comp

:Overlay
   .Curve.Scaled          :Curve   [k]   (inertia)
   .Curve.Scaled_plus_PCA :Curve   [k]   (inertia)

In [90]:
# Composite plot to contrast the cryptocurrency cluster graphs
crypto_plot + crypto_plot_pca

:Layout
   .NdOverlay.I  :NdOverlay   [k_Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [k_Cluster_PCA]
      :Scatter   [PC1]   (PC2,coin_id)

In [110]:
# Concatenate prediction data
concat_pred_df = pd.concat([transformed_prediction_df['k_Cluster'], pca_prediction_df['k_Cluster_PCA']], axis = 1)
diff_values = concat_pred_df[concat_pred_df['k_Cluster'] != concat_pred_df['k_Cluster_PCA']]

# Checking if diff_values (contains where k_Cluster does not equal k_Cluster_PCA) is empty or not. If empty, all predicted values are the same.
if len(diff_values) == 0:
    print("All predicted values are the same.")
else:
    print("Not all predicted values are the same.")
    diff_values

All predicted values are the same.


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using 3 features instead of 7 features did not make an impact on the elbow curve. As seen in the side-by-side comparisons of the elbow curves (scaled vs scaled + PCA), the results are the same. After applying PCA to the data and reducing to 3 features, 89.5% of the variance in the data is explained by these three features, and the predictions matched those of the clusters obtained from the prior method with 7 features.